In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from tensorflow.keras.callbacks import EarlyStopping
# from textpreprocessor import TextPreprocessor
import joblib, os
import pandas as pd
# Initialize the Text Pre Processor class
# processor = TextPreprocessor()

INPUT_DIR = "../Output/proto_models_rev2"
X_train_pad = joblib.load(os.path.join(INPUT_DIR, 'X_train_pad.pkl'))
X_test_pad = joblib.load(os.path.join(INPUT_DIR, 'X_test_pad.pkl'))

# Load data
df_train = pd.read_csv(os.path.join(INPUT_DIR, 'train_cleaned.csv'))
df_test = pd.read_csv(os.path.join(INPUT_DIR, 'test_cleaned.csv'))
# df_test = processor.load_data()

y_train = df_train['polarity']
y_test = df_test['polarity']

In [22]:
# Define Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor the validation loss
    patience=2,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore the weights of the best epoch
)

In [23]:
# RNN Model Building
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128))
model.add(SimpleRNN(64))  # Using a Simple RNN layer
model.add(Dense(1, activation='sigmoid'))  # Binary classification
model.summary()
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(X_train_pad, y_train, epochs=20, batch_size=256, validation_data=(X_test_pad, y_test), callbacks= early_stopping)
# Evaluate the model
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype("int32")
# Model evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_16 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_16 (SimpleRNN)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


I0000 00:00:1728595292.468859  126220 service.cc:146] XLA service 0x7fe17c0090d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728595292.469058  126220 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-10-10 23:21:32.497253: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-10 23:21:32.613572: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906


 3/40 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.4835 - loss: 0.7019

I0000 00:00:1728595293.175850  126220 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - accuracy: 0.5656 - loss: 0.6738 - val_accuracy: 0.7115 - val_loss: 0.5357
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.7919 - loss: 0.4641 - val_accuracy: 0.8120 - val_loss: 0.4002
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.9025 - loss: 0.2573 - val_accuracy: 0.8220 - val_loss: 0.3959
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.9634 - loss: 0.1366 - val_accuracy: 0.8255 - val_loss: 0.4278
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.9895 - loss: 0.0643 - val_accuracy: 0.8100 - val_loss: 0.4851
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
Model Accuracy: 82.20%
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.76      0.79       893
           1       0.82      0.87      0.84      1107

    accuracy                           0.82      2000
   macro avg       0.82      0.82      0.82      2000
weighted avg     

In [4]:
X_train_pad.shape

(10000, 222)

In [5]:
y_train.shape

(10000,)

In [6]:
X_test_pad.shape

(2000, 222)

In [7]:
# Define the model creation function
# Define the model creation function with **kwargs to accept arbitrary keyword arguments
def create_rnn_model(rnn_units=64, output_dim = 128):
    model = Sequential()
    model.add(Embedding(input_dim=5000, output_dim=output_dim))
    model.add(SimpleRNN(rnn_units))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Wrap the model with KerasClassifier, defining default hyperparameters here
rnn_model = KerasClassifier(model=create_rnn_model, output_dim=128, rnn_units=64, optimizer='adam', epochs=10, batch_size=512, verbose=1)

# Hyperparameter grid
param_dist = {
    'output_dim': [64, 128, 256],  # Embedding output dimension
    'rnn_units': [32, 64, 128],   # Number of RNN units
    'optimizer': ['adam', 'rmsprop']  # Optimizer
}

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Configure RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rnn_model,
    param_distributions=param_dist,
    n_iter=5,  
    scoring='accuracy',
    cv=3,
    verbose=1,
    n_jobs=1  # Set to 1 to avoid parallelism issues
)

# Execute the search
random_search.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), callbacks=[early_stopping])

# Display best parameters
print("Best parameters found by RandomizedSearchCV:", random_search.best_params_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


ValueError: 
All the 15 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/scikeras/wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/scikeras/wrappers.py", line 770, in fit
    self._fit(
  File "/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/scikeras/wrappers.py", line 936, in _fit
    self._check_model_compatibility(y)
  File "

In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

In [7]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding

# RNN 모델 정의
def build_rnn_model(input_dim, output_dim=1, rnn_units=32, embedding_dim=32, input_length=222, optimizer='adam'):
    model = Sequential()
    model.add(Embedding(input_dim=5000, output_dim=output_dim))
    model.add(SimpleRNN(rnn_units))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# KerasClassifier로 모델 래핑
input_dim = 5000  # 단어 사전 크기 (예시로 5000)
input_length = 222  # 문장의 최대 길이
model = KerasClassifier(
    build_fn=build_rnn_model,  # 모델 생성 함수
    input_dim=input_dim,
    input_length=input_length,
    verbose=1  # 훈련 과정 출력
)

# 하이퍼파라미터 검색 공간 정의
param_dist = {
    'rnn_units': [16, 32, 64],       # RNN 유닛 수
    'embedding_dim': [16, 32, 64],   # 임베딩 차원 수
    'optimizer': ['adam', 'rmsprop'], # 옵티마이저 선택
    'epochs': [5, 10],                # 에포크 수
    'batch_size': [16, 32]            # 배치 크기
}

# RandomizedSearchCV 설정
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=10,  # 시도할 하이퍼파라미터 조합 개수
    cv=3,       # 교차검증 folds 수
    verbose=1,
    n_jobs=2,  # 가능한 모든 CPU 코어 사용
    random_state=42
)

# 데이터 예시 (여기서는 이미 전처리된 X_train_pad와 y_train 사용)
random_search.fit(X_train_pad, y_train)

# 최적의 하이퍼파라미터와 모델 출력
print(f"Best parameters found: {random_search.best_params_}")
best_model = random_search.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/tmp/ipykernel_132321/1656335232.py:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(
2024-10-10 23:32:31.257520: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-10 23:32:31.278431: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-10 23:32:31.600027: W tensorflow/compiler/xla/st

Epoch 1/5
Epoch 1/5


2024-10-10 23:32:34.512817: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-10-10 23:32:34.609217: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f5b404d5dc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-10 23:32:34.609240: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-10-10 23:32:34.611795: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-10 23:32:34.649512: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


  3/417 [..............................] - ETA: 1:34 - loss: 0.6939 - accuracy: 0.5000

...

105/105 [==============================] - 9s 84ms/step - loss: 0.5854 - accuracy: 0.7966
Epoch 1/10
209/209 [==============================] - 133s 627ms/step - loss: 0.6602 - accuracy: 0.6024
Epoch 2/10
209/209 [==============================] - 125s 586ms/step - loss: 0.6770 - accuracy: 0.5685
Epoch 2/10
209/209 [==============================] - 114s 547ms/step - loss: 0.4833 - accuracy: 0.7779
Epoch 3/10
209/209 [==============================] - 113s 542ms/step - loss: 0.5251 - accuracy: 0.7600
Epoch 3/10
209/209 [==============================] - 116s 546ms/step - loss: 0.3606 - accuracy: 0.8473
Epoch 4/10
209/209 [==============================] - 110s 529ms/step - loss: 0.3741 - accuracy: 0.8479
Epoch 4/10
209/209 [==============================] - 115s 552ms/step - loss: 0.2818 - accuracy: 0.8862
Epoch 5/10
209/209 [==============================] - 110s 529ms/step - loss: 0.2660 - accuracy: 0.8994
Epoch 5/10
209/209 [==============================] - 113s 543ms/step - loss: 

/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
2024-10-11 05:18:09.978574: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-11 05:18:10.004126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-11 05:18:10.004162: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


Epoch 1/10


2024-10-11 05:18:10.005071: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-11 05:18:10.007823: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-11 05:18:10.007849: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-11 05:18:10.007861: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_nod

625/625 [==============================] - 92s 146ms/step - loss: 0.6666 - accuracy: 0.5829
Epoch 2/10
625/625 [==============================] - 89s 143ms/step - loss: 0.4793 - accuracy: 0.7810
Epoch 3/10
625/625 [==============================] - 90s 143ms/step - loss: 0.3707 - accuracy: 0.8336
Epoch 4/10
625/625 [==============================] - 90s 143ms/step - loss: 0.3215 - accuracy: 0.8632
Epoch 5/10
625/625 [==============================] - 89s 143ms/step - loss: 0.2970 - accuracy: 0.8737
Epoch 6/10
625/625 [==============================] - 90s 143ms/step - loss: 0.2819 - accuracy: 0.8831
Epoch 7/10
625/625 [==============================] - 90s 143ms/step - loss: 0.2708 - accuracy: 0.8884
Epoch 8/10
625/625 [==============================] - 90s 143ms/step - loss: 0.2637 - accuracy: 0.8905
Epoch 9/10
625/625 [==============================] - 90s 144ms/step - loss: 0.2593 - accuracy: 0.8956
Epoch 10/10
625/625 [==============================] - 90s 144ms/step - loss: 0.2599

In [12]:
def create_cnn_model(filters=32, kernel_size=3, pool_size=2, dropout_rate=0.25, learning_rate=0.001):
    model = Sequential([
        Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(222, 1)),
        MaxPooling1D(pool_size=pool_size),
        Dropout(dropout_rate),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# CNN 모델을 KerasClassifier로 래핑
model = KerasClassifier(model=create_cnn_model, epochs=10, batch_size=32, verbose=0)

# 하이퍼파라미터 범위 설정
param_dist = {
    'model__filters': [16, 32, 64],
    'model__kernel_size': [3, 5],
    'model__pool_size': [2, 3],
    'model__dropout_rate': [0.25, 0.5],
    'model__learning_rate': [0.001, 0.0001],
    'batch_size': [32, 64],
    'epochs': [10, 20]
}

# 입력 데이터의 차원 확장
X_train_cnn = np.expand_dims(X_train_pad, axis=2)

# RandomizedSearchCV 설정
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=20, cv=3, verbose=1, n_jobs=4, random_state=42)

# 모델 학습
random_search_result = random_search.fit(X_train_cnn, y_train)

# 최적 하이퍼파라미터 출력
print(f"Best parameters found: {random_search_result.best_params_}")
print(f"Best accuracy: {random_search_result.best_score_:.2f}")

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first

Best parameters found: {'model__pool_size': 3, 'model__learning_rate': 0.001, 'model__kernel_size': 3, 'model__filters': 64, 'model__dropout_rate': 0.25, 'epochs': 10, 'batch_size': 64}
Best accuracy: 0.53
